<!--INFORMATION-->

Written by IALS - 25.10.2024

>> *This notebook is based on  the [EEwPython course](https://colab.research.google.com/github/csaybar/EEwPython/blob/master/index.ipynb) as well as on the [Google Earth Engine Guides](https://developers.google.com/earth-engine/).

<center>
<h1>Google Earth Engine with Python </h1>
<h2> GEE Image </h2>
</center>
<h2> Topics:</h2>

1. Image Overview
2. Image Visualization
3. Image information and metadata
4. Mathematical Expressions
5. Relational, conditional and Boolean operations
6. Convolutions
7. Morphological Operations
8. Gradients
9. Edge Detection
10. Spectral Transformations
11. Texture
12. Object-based Methods
13. Cumulative Cost Mapping
14. Registering Images

# Images in GEE 

Raster data are represented as Image objects in Earth Engine. Images are composed of one or more bands and each band has its own name, data type, scale, mask and projection. Each image can have metadata stored as a set of properties (Dictionary). This section explains how to work with GEE images using Python.

First of all, make sure you have installed the [earthengine_api](https://developers.google.com/earth-engine/guides/python_install) as well as  [folium](https://python-visualization.github.io/folium/latest/) in your Python environment. Then, run the following cells to import and initialize the GEE Python API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [1]:
# Import the earthengine-api
import ee

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

True

In [3]:
# Initialize the library.
# Change name of Google Cloud project
ee.Initialize(project='ee-ivanlizarazo')

Now, let's import folium:

In [4]:
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

Folium version: 0.18.0


Now, let's create a function to display GEE objects using *folium*:

In [5]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

# 1. Image Overview



In addition to loading images from the archive by an **image ID**, you can also create images from constants, lists or other suitable Earth Engine objects. This section illustrates methods for creating images, getting band subsets, and manipulating bands.

Methods that you will use in this seccion:
 - **ee.Image.cat**
 - **ee.Image.addBands**
 - **ee.Image.Select**

In [6]:
# 2.1 Create a constant image.
image1 = ee.Image(5)
image1.getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 5, 'max': 5},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

- **ee.Image.cat(list)**: Concatenate the given images together into a single image.

  

In [7]:
#2.2 Concatenate two images into one multi-band image.
image2 = ee.Image(8)
image3 = ee.Image.cat([image1, image2])
image3.getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 5, 'max': 5},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'constant_1',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 8, 'max': 8},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [8]:
# Create a multi-band image from a list of constants.
multiband = ee.Image([1, 2, 3])
multiband.getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'constant_1',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'constant_2',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [9]:
# Select and (optionally) rename bands.
renamed = multiband.select(['constant', 'constant_1', 'constant_2'],
                           ['band1', 'band2', 'band3'])
renamed.getInfo()

{'type': 'Image',
 'bands': [{'id': 'band1',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'band2',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 2, 'max': 2},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'band3',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 3, 'max': 3},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

### Bands addition

- **ee.Image.addBands**: Returns an image containing all bands copied from the first input and selected bands from the second input, optionally overwriting bands in the first image with the same name. The new image has the metadata and footprint from the first input image

In [10]:
# Add bands to an image.
image4 = image3.addBands(ee.Image(42))
image4.getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 5, 'max': 5},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'constant_1',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 8, 'max': 8},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'constant_2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 42,
    'max': 42},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

# 2. Image Vizualization

The Python API  a difference than the Code Editor do not support interactive vizualization (**Map.addLayer**). To achieve desirable visualization effects you could use **ee.Image.getThumbURL** in replacement of Map.addLayer. Both  ee.Image.getThumbURL and Map.addLayer have the same parameters:



Parameter | Description | Type
--------------|-------------|-------------
bands	| Comma-delimited list of three band names to be mapped to RGB| list
min|	Value(s) to map to 0	number or list of three numbers, one for each |band
max|	Value(s) to map to 255	|number or list of three numbers, one for each band
gain|	Value(s) by which to multiply each pixel value|	number or list of three numbers, one for each band
bias|	Value(s) to add to each DN	| number or list of three numbers, one for each band
gamma|	Gamma correction factor(s)	| number or list of three numbers, one for each band
palette|	List of CSS-style color strings (single-band images only)	|comma-separated list of hex strings
opacity|	The opacity of the layer (0.0 is fully transparent and 1.0 is fully opaque)	| number
format|	Either "jpg" or "png"	|string

Next, we will look for a USGS Landsat 9 Collection 2 Tier 1 TOA Reflectance images collected in October 2024 over Bogota capital city of Colombia.  For more information on this collection see [here](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T2_TOA).

In [11]:
geom = ee.Geometry.Point(-74.8, 10.5)
col = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterDate('2024-01-01','2024-10-30').filterBounds(geom).filter(ee.Filter.lt('CLOUD_COVER', 5));


In [12]:
# Load the first image in the collection
image =  col.first()

In [13]:
# What is the ID of the image?
# 

In [14]:
# get MapId
mapid = image.getMapId({
    'bands': ['B4', 'B3', 'B2'], 
    'min': 0.05, 
    'max': 0.15})
# Define center
center=[10.5, -74.8]
# Display the map with folium!
Mapdisplay(center, {'Landsat image':mapid},zoom_start=10)

## Masking

You can use **`image.updateMask()`** to set the opacity of individual pixels based on where pixels in a mask image are non-zero. Pixels equal to zero in the mask are excluded from computations and the opacity is set to 0 for display. The following example uses an NDWI threshold (see the [Relational Operations section]() for information on thresholds) to update the mask on the NDWI layer created previously:



In [15]:
# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])

# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))

ndwiId = ndwiMasked.getMapId({'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']})
# Display the map with folium
# Define center
center=[10.5, -74.8]
Mapdisplay(center,{'NDWI masked':ndwiId})

## Clipping

The **`image.clip()`** method is useful for achieving cartographic effects. The following example clips the image.

In [16]:
# Create a circle by drawing a 50000 meter buffer around a point.
roi = ee.Geometry.Point([-74.8, 10.5]).buffer(20000)

mapRoi = image.clip(roi)

mapId = mapRoi.getMapId({'bands': ['B5', 'B6', 'B4'], 'min': 0.05, 'max': 0.35})

# Define center
center=[10.5, -74.8]
Mapdisplay(center,{'Mosaic':mapId}, zoom_start=11)

## Rendering categorical maps

Palettes are also useful for rendering discrete valued maps, for example a land cover map. In the case of multiple classes, use the palette to supply a different color for each class. (The **`image.remap()`** method may be useful in this context, to convert arbitrary labels to consecutive integers). The following example uses a palette to render land cover categories:



In [17]:
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01').select('Land_Cover_Type_1')

# Define a palette for the 18 distinct land cover classes.
igbpPalette = [
  'aec3d4', # water
  '152106', '225129', '369b47', '30eb5b', '387242', # forest
  '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # shrub, grass
  '111149', # wetlands
  'cdb33b', # croplands
  'cc0013', # urban
  '33280d', # crop mosaic
  'd7cdcc', # snow and ice
  'f7e084', # barren
  '6f6f6f'  # tundra
]

# Specify the min and max labels and the color palette matching the labels.
vizParams = {'min': 0,
             'max': 17,
             'palette': igbpPalette}
mapId = cover.getMapId(vizParams)

#Display the map with folium!
center=[10.5, -74.8]
Mapdisplay(center,{'Mosaic':mapId},zoom_start=11)

# 3. Image information and metadata

To explore image bands and properties in the Python API, getInfo() the image and inspect the output in the console. This information can also be accessed programmatically. For example, the following demonstrates how to access information about bands, projections and other metadata:




In [18]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_009053_20240104')

# Get information about the bands as a list.
bandNames = image.bandNames()
print('Band names: ', bandNames.getInfo()) # ee.List of band names

Band names:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'QA_PIXEL', 'QA_RADSAT', 'SAA', 'SZA', 'VAA', 'VZA']


In [19]:
# Get projection information from band 1.
b1proj = image.select('B1').projection()
print('Band 1 projection: \n', b1proj.getInfo()) # ee.Projection object

Band 1 projection: 
 {'type': 'Projection', 'crs': 'EPSG:32618', 'transform': [30, 0, 412185, 0, -30, 1234515]}


In [20]:
# Get scale (in meters) information from band 1.
b1scale = image.select('B1').projection().nominalScale()
print('Band 1 scale: \n', b1scale.getInfo()) # ee.Number

Band 1 scale: 
 30


In [21]:
# Note that different bands can have different projections and scale.
b8scale = image.select('B8').projection().nominalScale()
print('Band 8 scale: \n', b8scale.getInfo()) # ee.Number

Band 8 scale: 
 15


In [22]:
# Get a list of all metadata properties.
properties = image.propertyNames()
print('Metadata properties:', ) 
properties.getInfo() # ee.List of metadata properties

Metadata properties:


['system:version',
 'system:id',
 'RADIANCE_MULT_BAND_5',
 'RADIANCE_MULT_BAND_6',
 'RADIANCE_MULT_BAND_3',
 'RADIANCE_MULT_BAND_4',
 'RADIANCE_MULT_BAND_1',
 'RADIANCE_MULT_BAND_2',
 'WRS_TYPE',
 'K2_CONSTANT_BAND_11',
 'K2_CONSTANT_BAND_10',
 'system:footprint',
 'REFLECTIVE_SAMPLES',
 'SUN_AZIMUTH',
 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION',
 'DATE_ACQUIRED',
 'ELLIPSOID',
 'STATION_ID',
 'RESAMPLING_OPTION',
 'ORIENTATION',
 'WRS_ROW',
 'RADIANCE_MULT_BAND_9',
 'TARGET_WRS_ROW',
 'RADIANCE_MULT_BAND_7',
 'RADIANCE_MULT_BAND_8',
 'IMAGE_QUALITY_TIRS',
 'TRUNCATION_OLI',
 'CLOUD_COVER',
 'GEOMETRIC_RMSE_VERIFY',
 'COLLECTION_CATEGORY',
 'GRID_CELL_SIZE_REFLECTIVE',
 'CLOUD_COVER_LAND',
 'GEOMETRIC_RMSE_MODEL',
 'COLLECTION_NUMBER',
 'IMAGE_QUALITY_OLI',
 'LANDSAT_SCENE_ID',
 'WRS_PATH',
 'PANCHROMATIC_SAMPLES',
 'PANCHROMATIC_LINES',
 'GEOMETRIC_RMSE_MODEL_Y',
 'REFLECTIVE_LINES',
 'GEOMETRIC_RMSE_MODEL_X',
 'system:asset_size',
 'system:index',
 'DATA_SOURCE_ELEVATION',
 'REFLECTAN

In [23]:
# Get a specific metadata property.
cloudiness = image.get('CLOUD_COVER')
print('CLOUD_COVER: ')
cloudiness.getInfo() # ee.Number

CLOUD_COVER: 


3.12

In [24]:
import datetime
from datetime import datetime as dt

# Get the timestamp and convert it to a date.
date = ee.Date(image.get('system:time_start'))
# We divide by 1000 because Earth Engine returns the timestamp in milisecond and Python in seconds.
tmstp = date.getInfo()['value']/1000 

print('Timestamp:', dt.fromtimestamp(tmstp, datetime.UTC).strftime('%Y-%m-%d %H:%M:%S'))
#print('Timestamp:', dt.utcfromtimestamp(tmstp).strftime('%Y-%m-%d %H:%M:%S'))

Timestamp: 2024-01-04 15:17:24


Note that the results of these queries are **server-side objects**. When you use the method \*.getInfo(), you request that information describing the object be sent from the server to your client. (Learn more about client vs. server in Earth Engine on this [page](https://developers.google.com/earth-engine/client_server)). 



# 4. Mathematical Expressions

Earth Engine supports many basic mathematical operators. They share some common features. **Earth Engine performs math operations per pixel**. When an operator is applied to an image, it's applied to each unmasked pixel of each band. In the case of operations on two images, the operation is only applied at the locations where pixels in both images are unmasked. Earth Engine automatically matches bands between images. When an operator is applied to two images, the images are expected to have the same number of bands so they can be matched pairwise. However, if one of the images has only a single band, it is matched with all of the bands in the other image, essentially replicating that band enough times to match the other image.


For a simple example, consider the task of creating the Normalized Difference Vegetation Index (NDVI) using Landsat imagery:



In [25]:
# Load two 5-year Landsat 7 composites.
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Compute NDVI the hard way.
ndvi1999 = landsat1999.select('B4')\
                      .subtract(landsat1999.select('B3'))\
                      .divide(landsat1999.select('B4').add(landsat1999.select('B3')))

# Compute NDVI the easy way.
ndvi2008 = landsat2008.normalizedDifference(['B4', 'B3'])

#Vizualization
ndwiViz = {'min': 0, 'max': 1, 'palette': ['FF0000', '00FF00']}
COL = ee.Geometry.Rectangle(-84, -5, -66,13) 

ndvi1999 = ndvi1999.clip(COL)
ndvi2008 = ndvi2008.clip(COL)

tiles = {
    "NDVI COLOMBIA 1999": ndvi1999.getMapId(ndwiViz), 
    "NDVI COLOMBIA 2008": ndvi2008.getMapId(ndwiViz)
}

center = [10.5, -74.]
Mapdisplay(center, tiles, zoom_start=9)

####  Expressions

To implement more complex mathematical expressions, it may be more convenient to use **`image.expression()`**, which parses a text representation of a math operation. The following example uses **`expression()`** to compute the Enhanced Vegetation Index (EVI):

In [26]:
# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_009053_20240104')

# Compute the EVI using an expression.
evi = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')})

#Display the map with folium!
Eviviz = {'min': 0, 'max': 1, 'palette': ['FF0000', '00FF00']}
tiles = {"EVI 2024": evi.getMapId(Eviviz)}

center  = [10.5,-74.8]
Mapdisplay(center, tiles)

Observe that the first argument to expression is the textual representation of the math operation, the second argument is a dictionary where the keys are variable names used in the expression and the values are the image bands to which the variables should be mapped. Bands in the image may be referred to as `b("band name")` or `b(index)`, for example `b(0)`, instead of providing the dictionary. Note that division functions as it does in Python: dividing two integers results in an integer. For example `10 / 20 = 0`. To change this behavior, multiply one of the operands by `1.0: 10 * 1.0 / 20 = 0.5`. Supported expression operators are listed in the following table.

 _ | _ | Operators for expression()
--------------|-------------|-------------
Arithmetic	|+ - * / % **	|Add, Subtract, Multiply, Divide, Modulus, Exponent
Comparison	| == != < > <= >= |	Equal, Not Equal, Less Than, Greater than, etc.
Logical |	&& \|\| ! ^	| And, Or, Not, Xor
Ternary	| ? :	| If then else

# 5. Relational, conditional and Boolean operations

To perform per-pixel comparisons between images, use relational operators. To extract urbanized areas in an image, this example uses relational operators to threshold spectral indices, combining the thresholds with and():

In [27]:
# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_009053_20240104')

# Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# Create a binary layer using logical operations.
bare = ndvi.lt(0.25).And(ndwi.lt(0))

bare = bare.updateMask(bare)
BareId = bare.getMapId({'min': 0, 'max': 1, 'palette': ['FFFFFF', '00FFFF']})

tiles = {"Bare": BareId}
center = [10.85, -74.8]
Mapdisplay(center, tiles, zoom_start=12)

As illustrated by this example, the output of relational and boolean operators is either true (1) or false (0). To mask the 0's, you can mask the resultant binary image with itself. The result from the previous example should look something.



The binary images that are returned by relational and boolean operators can be used with mathematical operators. This example creates zones of urbanization in a nighttime lights image using relational operators and image.add():



In [28]:
# Load a 2012 nightlights image.
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl2012.select('stable_lights')

# Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

# Display the thresholded image as three distinct zones in the Colombian Caribbean Coast.
palette = ['000000', '0000FF', '00FF00', 'FF0000']
center = [10.85, -74.8]

zonesId = zones.getMapId({'min': 0, 'max': 3, 'palette': palette})
tiles = {"Zonas": zonesId}

Mapdisplay(center, tiles, zoom_start=10)

Note that the code in the previous example is equivalent to using a ternary operator implemented by expression()


In [29]:
zonesExp = nl2012.expression("(b('stable_lights') > 62) ? 3: (b('stable_lights') > 55) ? 2: (b('stable_lights') > 30) ? 1: 0")

zonesId = zonesExp.getMapId({'min': 0, 'max': 3, 'palette': palette})
tiles = {"Zonas": zonesId}

Mapdisplay(center, tiles)

Another way to implement conditional operations on images is with the image.where() operator. Consider the need to replace masked pixels with some other data. In the following example, cloudy pixels are replaced by pixels from a cloud-free image using where()

In [30]:
# Load a cloudy Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_009053_20240308')

# Load another image to replace the cloudy pixels.
replacement = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_009053_20240104')


# Compute a cloud score band.
cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')

# Set cloudy pixels to the other image.
replaced = image.where(cloud.gt(10), replacement)

# Display the result.
cloudId = image.getMapId({'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5})
replacedId = replaced.getMapId({'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5})

tiles0 = {"Clouds replaced": cloudId}
tiles1 = {"Clouds replaced": replacedId}

center = [10.5, -74.8]
# uncomment to see the cloudy image
#Mapdisplay(center, tiles0)
Mapdisplay(center, tiles1)

In case you are interested in saving an html file displaying the two images (i.e cloudy and clear), uncomment the next two chunks.  

In [31]:
# # import library
# import webbrowser
# # define method for displaying Earth Engine image tiles in folium maps
# def add_ee_layer(self, ee_image_object, vis_params, name):
#   map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
#   folium.raster_layers.TileLayer(
#       tiles = map_id_dict['tile_fetcher'].url_format,
#       attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#       name = name,
#       overlay = True,
#       control = True
#   ).add_to(self)

# # adding EE drawing method to folium
# folium.Map.add_ee_layer = add_ee_layer

In [32]:
# vis = {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5}

# m = folium.Map(location=[10.5, -74.8], zoom_start=10)
# basemap_dark = folium.TileLayer('cartodbdark_matter', name='Dark Matter')
# basemap_dark.add_to(m)
# m.add_ee_layer(image, vis, 'cloudy')
# m.add_ee_layer(replaced, vis, 'clear')
# ##################
# # adding layer controller 
# folium.LayerControl(collapsed=False).add_to(m)

# m.save('a-map.html')
# webbrowser.open('a-map.html')


In this example, observe the use of the simpleCloudScore() algorithm. This algorithm ranks pixels by cloudiness on a scale of 0-100, with 100 most cloudy. Learn more about simpleCloudScore() on the Landsat Algorithms page.

# 6. Convolutions

To perform linear convolutions on images, use **image.convolve()**. The only argument to convolve is an ee.Kernel which is specified by a shape and the weights in the kernel. Each pixel of the image output by **convolve()** is the linear combination of the kernel values and the input image pixels covered by the kernel. The kernels are applied to each band individually. For example, you might want to use a low-pass (smoothing) kernel to remove high-frequency information. The following illustrates a 15x15 low-pass kernel applied to a Landsat 8 image

In [33]:
# Load and display an image
image   = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_009053_20240104')

# Define a boxcar or low-pass kernel
boxcar  = ee.Kernel.square(7, 'pixels', True)

# Smooth the image by convolving with the boxcar kernel
smooth = image.convolve(boxcar)
  
tiles = {
    "input image": image.getMapId({'bands': ['B5', 'B4', 'B3'], 'max': 0.5}),
    "smoothed": smooth.getMapId({'bands': ['B5', 'B4', 'B3'], 'max': 0.5})
}

center = [10.5, -74.8]
Mapdisplay(center, tiles, zoom_start=11)

The output of convolution with the low-pass filter should look something like figure on top. Observe that the arguments to the kernel determine its size and coefficients. Specifically, with the units parameter set to pixels, the radius parameter specifies the number of pixels from the center that the kernel will cover. If normalize is set to true, the kernel coefficients will sum to one. If the magnitude parameter is set, the kernel coefficients will be multiplied by the magnitude (if normalize is also true, the coefficients will sum to magnitude). If there is a negative value in any of the kernel coefficients, setting normalize to true will make the coefficients sum to zero.

Use other kernels to achieve the desired image processing effect. This example uses a Laplacian kernel for isotropic edge detection

In [34]:
# Define a Laplacian, or edge-detection kernel.
laplacian = ee.Kernel.laplacian8(normalize = False)

# Apply the edge-detection kernel.
edgy = image.convolve(laplacian)
  
tiles = {"edges": edgy.getMapId({'bands': ['B5', 'B6', 'B4'], 'max': 0.5})}

center = [10.5, -74.8]
Mapdisplay(center, tiles)

Note the format specifier in the visualization parameters. Earth Engine sends display tiles to the Code Editor in JPEG format for efficiency, however edge tiles are sent in PNG format to handle transparency of pixels outside the image boundary. When a visual discontinuity results, setting the format to PNG results in a consistent display. The result of convolving with the Laplacian edge detection kernel should look something like to figure on top.


There are also anisotropic edge detection kernels (e.g. Sobel, Prewitt, Roberts), the direction of which can be changed with kernel.rotate(). Other low pass kernels include a Gaussian kernel and kernels of various shape with uniform weights. To create kernels with arbitrarily defined weights and shape, use ee.Kernel.fixed(). For example, this code creates a 9x9 kernel of 1’s with a zero in the middle

In [35]:
from pprint import pprint
# Create a list of weights for a 9x9 kernel.
lista = [1, 1, 1, 1, 1, 1, 1, 1, 1]
# The center of the kernel is zero.
centerList = [1, 1, 1, 1, 0, 1, 1, 1, 1]
# Assemble a list of lists: the 9x9 kernel weights as a 2-D matrix.
lists = [lista, lista, lista, lista, centerList, lista, lista, lista, lista]
# Create the kernel from the weights.
kernel = ee.Kernel.fixed(9, 9, lists, -4, -4, False)
pprint(kernel.getInfo())

{'center': [4, 4],
 'height': 9,
 'type': 'Kernel.fixed',
 'weights': '\n'
            '  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]\n'
            '  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]\n'
            '  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]\n'
            '  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]\n'
            '  [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]\n'
            '  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]\n'
            '  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]\n'
            '  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]\n'
            '  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]',
 'width': 9,
 'x': 4,
 'y': 4}


# 7. Morphological Operations

Earth Engine implements morphological operations as focal operations, specifically focal_max(), focal_min(), focal_median(), and focal_mode() instance methods in the Image class. (These are shortcuts for the more general reduceNeighborhood(), which can input the pixels in a kernel to any reducer with a numeric output. See this page for more information on reducing neighborhoods). The morphological operators are useful for performing operations such as erosion, dilation, opening and closing. For example, to perform an opening operation, use focal_min() followed by focal_max().

In [36]:
# Load a Landsat 8 image, select the NIR band, threshold, display.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_009053_20240104').select(4).gt(0.2);


# Define a kernel.
kernel = ee.Kernel.circle(radius=1)

# Perform an erosion followed by a dilation, display.
opened = image.focal_min(kernel=kernel, iterations=2).focal_max(kernel=kernel, iterations=2)

tiles = {'NIR threshold': image.getMapId(),
         'opened'       : opened.getMapId()}

center = [10.5, -74.8]
Mapdisplay(center, tiles, zoom_start=11)

Note that in the previous example, a kernel argument is provided to the morphological operator. The pixels covered by non-zero elements of the kernel are used in the computation. The iterations argument indicates how many times to apply the operator.

# 8. Gradients

You can compute the gradient of each band of an image with image.gradient(). For example, the following code computes the gradient magnitude and direction of the Landsat 8 panchromatic band

In [37]:
# Load a Landsat 8 image and select the panchromatic band.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_009053_20240104').select('B8');

# Compute the image gradient in the X and Y directions.
xyGrad = image.gradient();

# Compute the magnitude of the gradient.
gradient = xyGrad.select('x').pow(2).add(xyGrad.select('y').pow(2)).sqrt();

# Compute the direction of the gradient.
direction = xyGrad.select('y').atan2(xyGrad.select('x'));

# Display the results.
tiles = {
    'direction': direction.getMapId({'min': -2, 'max': 2}),
    'gradient' : opened.getMapId({'min': -7, 'max': 7})
}

center = [10.5, -74.8]
Mapdisplay(center, tiles, zoom_start=11)

# 9. Edge Detection

Edge detection is applicable to a wide range of image processing tasks. In addition to the edge detection kernels described in the convolutions section, there are several specialized edge detection algorithms in Earth Engine. The Canny edge detection algorithm (Canny 1986) uses four separate filters to identify the diagonal, vertical, and horizontal edges. The calculation extracts the first derivative value for the horizontal and vertical directions and computes the gradient magnitude. Gradients of smaller magnitude are suppressed. To eliminate high-frequency noise, optionally pre-filter the image with a Gaussian kernel. 



In [38]:
# Load a Landsat 8 image, select the panchromatic band.
image = ee.Image('LANDSAT/LC08/C02/T1/LC08_044034_20140318').select('B8')

# Perform Canny edge detection and display the result.
canny = ee.Algorithms.CannyEdgeDetector(image, threshold= 10, sigma= 1)

tiles = {'Original': image.getMapId(),
         'Canny Edges'       : canny.getMapId()}

# Display
center = [37.7, -122.1]
Mapdisplay(center, tiles, zoom_start=11)

Note that the threshold parameter determines the minimum gradient magnitude and the sigma parameter is the standard deviation (SD) of a Gaussian pre-filter to remove high-frequency noise. For line extraction from an edge detector, Earth Engine implements the Hough transform (Duda and Hart 1972). Now extract lines from the Canny detector.

In [39]:
# Perform Hough transform of the Canny result and display.
hough = ee.Algorithms.HoughTransform(canny, 256, 600, 100);

# Display the results.
tiles = {'hough': hough.getMapId()}

center = [37.7, -122.1]
Mapdisplay(center, tiles, zoom_start=11)

Another specialized algorithm in Earth Engine is zeroCrossing(). A zero-crossing is defined as any pixel where the right, bottom, or diagonal bottom-right pixel has the opposite sign. If any of these pixels is of opposite sign, the current pixel is set to 1 (zero-crossing); otherwise it's set to zero. To detect edges, the zero-crossings algorithm can be applied to an estimate of the image second derivative. The following demonstrates using zeroCrossing() for edge detection.

In [40]:
# Load a Landsat 8 image, select the panchromatic band.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_009053_20240104').select('B8')


# Define a "fat" Gaussian kernel.
fat = ee.Kernel.gaussian(
  radius=3,
  sigma=3,
  units='pixels',
  normalize=True,
  magnitude=-1
)

# Define a "skinny" Gaussian kernel.
skinny = ee.Kernel.gaussian(
  radius=3,
  sigma=1,
  units='pixels',
  normalize=True,
)

# Compute a difference-of-Gaussians (DOG) kernel.
dog = fat.add(skinny)

# Compute the zero crossings of the second derivative, display.
zeroXings = image.convolve(dog).zeroCrossing()

# Display the results.
tiles = {
    'image': image.getMapId({'max': 12000}),
    'zero crossings': zeroXings.updateMask(zeroXings).getMapId({'palette': 'FF0000'})
}

center = [10.5, -74.8]
Mapdisplay(center, tiles, zoom_start=11)

Zero-crossings output (red) with the Landsat 8 panchromatic band in the background for an area near the San Francisco, California airport (right).

# 10. Spectral Transformations

There are several spectral transformation methods in Earth Engine. These include instance methods on images such as normalizedDifference(), unmix(), rgbToHsv() and hsvToRgb(). The latter two methods are useful for pan sharpening.

In [41]:
# Load a Landsat 8 top-of-atmosphere reflectance image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_009053_20240104')

# Convert the RGB bands to the HSV color space.
hsv = image.select(['B4', 'B3', 'B2']).rgbToHsv()

# Swap in the panchromatic band and convert back to RGB.
sharpened = ee.Image.cat([
  hsv.select('hue'), hsv.select('saturation'), image.select('B8')
]).hsvToRgb()

# Display the results.
tiles = {
    'rgb': image.getMapId({'bands': ['B4', 'B3', 'B2'], 
                           'min': 0, 
                           'max': 0.25, 
                           'gamma': [1.1, 1.1, 1]}),
    'pan-sharpened': sharpened.getMapId({'min': 0, 
                                         'max': 0.25, 
                                         'gamma': [1.3, 1.3, 1.3]})
}

center = [10.5, -74.8]
Mapdisplay(center, tiles, zoom_start=11)

Spectral unmixing is implemented in Earth Engine as the image.unmix() method. (For more flexible methods, see the Array Transformations page). The following is an example of unmixing Landsat 5 with predetermined urban, vegetation and water endmembers.

In [42]:
# Load a Landsat 5 image and select the bands we want to unmix.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'];
image = ee.Image('LANDSAT/LC08/C02/T1/LC08_044034_20140318').select(bands)

# Define spectral endmembers.
urban = [88, 42, 48, 38, 86, 115, 59]
veg = [50, 21, 20, 35, 50, 110, 23]
water = [51, 20, 14, 9, 7, 116, 4]

# Unmix the image.
fractions = image.unmix([urban, veg, water])

# Display the results.
tiles = {
    'image': image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 128}),
    'unmixed': fractions.getMapId()
}

center = [37.7, -122.1]
Mapdisplay(center, tiles, zoom_start=11)

The unmixing result should look something like figure on top.
Landsat 5 imagery unmixed to urban (red), vegetation (green) and water (blue) fractions. San Francisco bay area, California, USA.

# 11. Texture

Earth Engine has several special methods for estimating spatial texture. When the image is discrete valued (not floating point), you can use image.entropy() to compute the entropy in a neighborhood

In [43]:
# Load a high-resolution NAIP image.
image = ee.Image('USDA/NAIP/DOQQ/m_3712213_sw_10_1_20140613')

# Get the NIR band.
nir = image.select('N')

# Define a neighborhood with a kernel.
square = ee.Kernel.square(radius=4)

# Compute entropy and display.
entropy = nir.entropy(square)

# Display the results.
tiles = {
    'image': image.getMapId({'max': 255}),
    'entropy': entropy.getMapId({'min': 1, 'max': 5, 'palette': ['0000CC', 'CC0000']})
}

center = [37.769833, -122.466123]
Mapdisplay(center, tiles, zoom_start=12)

Note that the NIR band is scaled to 8-bits prior to calling entropy() since the entropy computation takes discrete valued inputs. The non-zero elements in the kernel specify the neighborhood.

Another way to measure texture is with a gray-level co-occurrence matrix (GLCM). Using the image and kernel from the previous example, compute the GLCM-based contrast as follows.

In [44]:
# Compute the gray-level co-occurrence matrix (GLCM), get contrast.
glcm = nir.glcmTexture(size=4)
contrast = glcm.select('N_contrast')

# Display the results.
tiles = {
    'contrast': contrast.getMapId({'min': 0, 'max': 1500, 'palette': ['0000CC', 'CC0000']})
}

center = [37.769833, -122.466123]
Mapdisplay(center, tiles, zoom_start=13)

Many measures of texture are output by image.glcm(). For a complete reference on the outputs, see Haralick et al. (1973) and Conners et al. (1984).

Local measures of spatial association such as Geary’s C (Anselin 1995) can be computed in Earth Engine using image.neighborhoodToBands(). 

In [45]:
import math
# Create a list of weights for a 9x9 kernel.
lista = [1, 1, 1, 1, 1, 1, 1, 1, 1]
# The center of the kernel is zero.
centerList = [1, 1, 1, 1, 0, 1, 1, 1, 1]
# Assemble a list of lists: the 9x9 kernel weights as a 2-D matrix.
lists = [lista, lista, lista, lista, centerList, lista, lista, lista, lista]
# Create the kernel from the weights.
# Non-zero weights represent the spatial neighborhood.
kernel = ee.Kernel.fixed(9, 9, lists, -4, -4, False)

# Convert the neighborhood into multiple bands.
neighs = nir.neighborhoodToBands(kernel)

# Compute local Geary's C, a measure of spatial association.
gearys = nir.subtract(neighs).pow(2).reduce(ee.Reducer.sum()).divide(math.pow(9, 2))

# Display the results.
tiles = {
    "Geary's C": gearys.getMapId({'min': 20, 'max': 2500, 'palette': ['0000CC', 'CC0000']})
}

center = [37.769833, -122.466123]
Mapdisplay(center, tiles, zoom_start=13)

For an example of using neighborhood standard deviation to compute image texture, see the [Statistics of Image Neighborhoods](https://developers.google.com/earth-engine/reducers_reduce_neighborhood) page.

# 12. Object-based Methods

For treating landscape elements as objects, Earth Engine contains several methods. Specifically, when an image has distinct patches identified by unique pixel values, use image.connectedPixelCount() to compute the number of pixels in each patch and image.connectedComponents() to label each patch with a unique identifier. The unique identifiers can then be used to enumerate the patches and analyze the distribution of size or some other quality of interest. The following computes the size and unique identifiers of hot patches in a surface temperature image.

In [46]:
# Load a Landsat 8 image and display the thermal band.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_009053_20240104')

# Threshold the thermal band to find "hot" objects.
hotspots = image.select('B10').gt(303)

# Mask "cold" pixels.
hotspots = hotspots.updateMask(hotspots)

# Compute the number of pixels in each patch.
patchsize = hotspots.connectedPixelCount(256, False)

# Uniquely label the patches and visualize.
patchid = hotspots.connectedComponents(ee.Kernel.plus(1), 256)
    
# Display the results.
tiles = {
    "LST": image.getMapId({'bands': ['B10'], 'min': 270, 'max': 310}),
    "hotspots": hotspots.getMapId({'palette': 'FF0000'}),
    "patchsize": patchsize.getMapId(),
    "patches": patchid.getMapId()
}

center = [10.90, -74.8]
Mapdisplay(center, tiles)

In the previous example, note that the maximum patch size is set to 256 pixels by the arguments to the connectedPixelCount() and connectedComponents() methods. The connectivity is also specified by the arguments, in the former method by a boolean and in the latter method by an ee.Kernel. In this example, only four neighbors are considered for each pixel.



# 13. Cumulative Cost Mapping

Use image.cumulativeCost() to compute a cost map where every pixel contains the total cost of the lowest cost path to the nearest source location. This process is useful in a variety of contexts such as habitat analysis (Adriaensen et al. 2003), watershed delineation (Melles et al. 2011) and image segmentation (Falcao et al. 2004). Call the cumulative cost function on an image in which each pixel represents the cost per meter to traverse it. Paths are computed through any of a pixel's eight neighbors. Required inputs include a source image, in which each non-zero pixel represents a potential source (or start of a path), and a maxDistance (in meters) over which to compute paths. The algorithm finds the cumulative cost of all paths less than maxPixels = maxDistance/scale in length, where scale is the pixel resolution, or scale of analysis in Earth Engine.


The following example demonstrates computing least-cost paths across a land cover image.


In [47]:
# A rectangle representing Bangui, Central African Republic.
geometry = ee.Geometry.Rectangle([18.5229, 4.3491, 18.5833, 4.4066])

# Create a source image where the geometry is 1, everything else is 0.
sources = ee.Image().toByte().paint(geometry, 1)

# Mask the sources image with itself.
sources = sources.updateMask(sources)

# The cost data is generated from classes in ESA/GLOBCOVER.
cover = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(0)

# Classes 60, 80, 110, 140 have cost 1.
# Classes 40, 90, 120, 130, 170 have cost 2.
# Classes 50, 70, 150, 160 have cost 3.
cost = cover.eq(60).Or(cover.eq(80)).Or(cover.eq(110)).Or(cover.eq(140)).multiply(1).add(cover.eq(40).Or(cover.eq(90)).Or(cover.eq(120)).Or(cover.eq(130)).Or(cover.eq(170)).multiply(2).add(cover.eq(50).Or(cover.eq(70)).Or(cover.eq(150)).Or(cover.eq(160)).multiply(3)))

# Compute the cumulative cost to traverse the land cover.
cumulativeCost = cost.cumulativeCost(
  source = sources,
  maxDistance = 80 * 1000 # 80 kilometers
)

dicc = {
    'Globcover': cover.getMapId(),
    'accumulated cost': cumulativeCost.getMapId({'min': 0, 'max': 5e4}),
    "source geometry": geometry.getInfo()
}

# Display the results
center = [4.2, 18.71]
Mapdisplay(center, dicc)

The top figure in which each output pixel represents the accumulated cost to the nearest source. Note that discontinuities can appear in places where the least cost path to the nearest source exceeds maxPixels in length.

The cumulative cost to the source pixels, where cost is determined by the land cover categories. Low costs are black, higher costs are white.

# 14. Registering Images

The Earth Engine image registration algorithm is designed to be a final, post-ortho, fine-grained step in aligning images. It is assumed that the images to be registered have already gone through initial alignment stages, so they are already within a few degrees of rotation of one another, and differ by only small translations. The registration uses a “rubber-sheet” technique, allowing local image warping to correct for orthorectification errors and other artifacts from earlier processing. The underlying alignment technique is image correlation, so the bands for the input and reference images must be visually similar in order for the algorithm to compute an accurate alignment.

## Image displacement

There are two steps to registering an image: Determining the displacement image using **displacement()**, and then applying it with **displace()**. The required inputs are the pair of images to register, and a maximum displacement parameter **(maxOffset)**.

The **displacement()** algorithm takes a reference image, a maximum displacement parameter **(maxOffset)**, and two optional parameters that modify the algorithm behaviour. The output is a displacement image with bands **dx** and **dy** which give the X and Y components (in meters) of the displacement vector at each pixel.

All bands of the calling and reference images are used for matching during registration, so the number of bands must be exactly equal. The input bands must be visually similar for registration to succeed. If that is not the case, it may be possible to pre-process them (e.g. smoothing, edge detection) to make them appear more similar. The registration computations are performed using a multiscale, coarse-to-fine process, with (multiscale) working projections that depend on three of the projections supplied to the algorithm:

1.   The default projection of the calling image (Pc)
2.   The default projection of the reference image (Pr)
3.   The output projection (Po)

The highest resolution working projection (*Pw* will be in the CRS of *Pr*, at a scale determined by the coarsest resolution of these 3 projections, to minimize computation. The results from Pr are then resampled to be in the projection specified by the input **'projection'** parameter.

The output is a displacement image with the following bands:

**dx:**

For a given reference image pixel location, this band contains the distance in the Y direction that must be travelled to arrive at the matching location in the calling image. Units are in geodesic meters.

**dy:**

For a given reference image pixel location, this band contains the distance in the Y direction that must be travelled to arrive at the matching location in the calling image. Units are in geodesic meters.

**confidence:**

This is a per-pixel estimate of displacement confidence (where 0 is low confidence and 1 is high confidence) based on the correlation scores in regions where valid matches were found. In regions where no matches were found, confidence is estimated from nearby correlations using a Gaussian kernel to provide higher weight to nearby correlations.

The following example computes the magnitude and angle of displacement between two high-resolution Terra Bella images:

In [48]:
# Load the two images to be registered.
image1 = ee.Image('SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL/s01_20150502T082736Z')
image2 = ee.Image('SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL/s01_20150305T081019Z')

# Use bicubic resampling during registration.
image1Orig = image1.resample('bicubic')
image2Orig = image2.resample('bicubic')

# Choose to register using only the 'R' band.
image1RedBand = image1Orig.select('R')
image2RedBand = image2Orig.select('R')

# Determine the displacement by matching only the 'R' bands.
displacement = image2RedBand.displacement(
  referenceImage = image1RedBand,
  maxOffset  = 50.0,
  patchWidth = 100.0
)

# Compute image offset and direction.
offset = displacement.select('dx').hypot(displacement.select('dy'))
angle = displacement.select('dx').atan2(displacement.select('dy'))

dicc = {
    'offset': offset.getMapId({'min':0, 'max': 20}),
    'angle': angle.getMapId({'min': -math.pi, 'max': math.pi})
}

# Display the results
center = [0.58, 37.44]
Mapdisplay(center, dicc,zoom_start= 12)

## Warping an image

There are two ways to warp an image to match another image: **displace()** or **register()**. The **displace()** algorithm takes a displacement image having **dx** and dy bands as the first two bands, and warps the image accordingly. The output image will be the result of warping the bands of the input image by the offsets present in the displacement image. Using the displacements computed in the previous example:

In [49]:
# Use the computed displacement to register all original bands.
registered = image2Orig.displace(displacement)

# Show the results of co-registering the images.
visParams = {'bands': ['R', 'G', 'B'], 'max': 4000}

dicc = {
    'Reference': image1Orig.getMapId(visParams),
    'Before Registration': image2Orig.getMapId(visParams),
    'After Registration':  registered.getMapId(visParams)
}

# Display the results
coords = [0.58, 37.44]
Mapdisplay(coords, dicc, zoom_start= 15)

If you don't need the displacement bands, Earth Engine provides the **register()** method, which is a shortcut for calling **displacement()** followed by **displace()**.

In [50]:
alsoRegistered = image2Orig.register(
  referenceImage = image1Orig,
  maxOffset = 50.0,
  patchWidth = 100.0
)

dicc = {
    'Also Registered': alsoRegistered.getMapId(visParams)
}

# Display the results
coords = [0.58, 37.44]
Mapdisplay(coords, dicc, zoom_start= 15)

In this example, the results of **register()** differ from the results of **displace()**. This is because a different set of bands was used in the two approaches: **register()** always uses all bands of the input images, while the **displacement()** example used only the red band before feeding the result to **displace()**. Note that when multiple bands are used, if band variances are very different this could over-weight the high-variance bands, since the bands are jointly normalized when their spatial correlation scores are combined. This illustrates the importance of selecting band(s) that are visually the most similar when registering. As in the previous example, use **displacement()** and **displace()** for control over which bands are used to compute displacement.